In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
import random
%matplotlib inline

In [ ]:
def f(x):
    return 4*x**2 - 6*x -9

In [ ]:
x = np.arange(-10,10,.5)
y = f(x)
plt.plot(x,y)

`f(x) = 4x2 - 6x - 9`

`f'(x) = 8x - 6`

In [ ]:
h = 0.0000001
x = 0.75
der = (f(x+h)-f(x))/h
print(der)

In [ ]:
# example case
a = 2
b = -3
c = 8

d = a*b + c

In [ ]:
a = 2
b = -3
c = 8

h = 0.00001
d1 = a*b + c
c+=h
d2 = a*b+c

print(d1)
print(d2)
print((d2-d1)/h)

# slope wrt c is .99
# slope wrt a is -3 , (since a is multiplied with a negative number)
# slope wrt b is 2

In [ ]:
class Value:
    # Holds a scalar value
    def __init__(self,data,_children=(),_op = '',label = ''):
        self.data = data
        self.grad = 0 # assumed
        self._backward = lambda: None
        self._prev = set(_children)
        self.label = label
        self._op = _op
        
    def __repr__(self):
        return f"value: {self.data}"
        
    def __add__(self,other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data+other.data,(self,other),'+')
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
        return out

    def __radd__(self, other):
        return self.__add__(other)
        
    def __mul__(self,other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data*other.data, (self,other),'*')
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out
        
    def __sub__(self,other):
        return self + (-other)

    def __neg__(self):
        return self*-1

    def __truediv__(self,other):
        return self * other ** -1

    def __pow__(self,other):
        assert isinstance(other, (int,float))
        out = Value(self.data**other, (self,),f'**{other}')
        def _backward():
            self.grad += other * (self.data**(other-1)) * out.grad
        out._backward = _backward
        return out

    def __rmul__(self,other): # other * self => 2 * a
        return self*other

    def exp(self):
        out = Value(math.exp(self.data), (self,),'exp')
        def _backward():
            self.grad += out.data * out.grad
        out._backward = _backward
        return out
        
    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
        out = Value(t,(self,),'tanh')
        def _backward():
            self.grad +=  out.grad * (1-t**2)
        out._backward = _backward
        return out
        

    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
          if v not in visited:
            visited.add(v)
            for child in v._prev:
              build_topo(child)
            topo.append(v)
        build_topo(self)
        # print(topo)
        
        self.grad = 1.0
        for node in reversed(topo):
          node._backward()

In [ ]:
## example

a = Value(2.0,label='a')
b = Value(-3.0,label='b')
c = Value(8.0, label = 'c')
e = a*b; e.label = 'e'
d = e + c; d.label = 'd'
f = Value(-2.0,label='f')
L = d*f


# Initialising all the gradients and doing grad checks
# L = d * f
L.grad = 1.0

# d and f grads 
d.grad = f.data
f.grad = d.data

# d = c + e
c.grad = d.grad * 1.0
e.grad = d.grad *1.0

# e = a * b
a.grad = e.grad * b.data
b.grad = e.grad * a.data

In [ ]:
## vis

from graphviz import Digraph

def trace(root):
  # builds a set of all nodes and edges in a graph
  nodes, edges = set(), set()
  def build(v):
    if v not in nodes:
      nodes.add(v)
      for child in v._prev:
        edges.add((child, v))
        build(child)
  build(root)
  return nodes, edges

def draw_dot(root):
  dot = Digraph(format='svg', graph_attr={'rankdir': 'LR', 'nodesep': '0.1', 'ranksep': '0.3'},node_attr={'fontsize': '8'})
  
  nodes, edges = trace(root)
  for n in nodes:
    uid = str(id(n))
    # for any value in the graph, create a rectangular ('record') node for it
    dot.node(name = uid, label = "{ %s | data %.3f | grd %.3f  }" % (n.label, n.data, n.grad), shape='record')
    if n._op:
      # if this value is a result of some operation, create an op node for it
      dot.node(name = uid + n._op, label = n._op)
      # and connect this node to it
      dot.edge(uid + n._op, uid)

  for n1, n2 in edges:
    # connect n1 to the op node of n2
    dot.edge(str(id(n1)), str(id(n2)) + n2._op)

  return dot

In [ ]:
draw_dot(L)

## 

In [ ]:
## gradient check

def numer_grad():
    h= 0.001
    a = Value(2.0,label='a')
    b = Value(-3.0,label='b')
    c = Value(10.0, label = 'c')
    e = a*b; e.label = 'e'
    d = e + c; d.label = 'd'
    f = Value(-2.0,label='f')
    L = d*f
    L1 = L.data

    a = Value(2.0,label='a')
    b = Value(-3.0,label='b')
    c = Value(10.0, label = 'c')
    e = a*b; e.label = 'e'
    d = e + c; d.label = 'd'
    f = Value(-2.0+h,label='f')
    L = d*f
    L2 = L.data

    print((L2 - L1)/h)

print('analyticaly dL/df = d = 4')
print('numeric grad wrt f: ',end='')
numer_grad()

In [ ]:
## 2D Neuron

# Inputs no activation function
x1 = Value(2.0,label = 'x1')
x2 = Value(0.0,label = 'x2')
w1 = Value(-3.0,label = 'w1')
w2 = Value(1.0,label = 'w2')
b = Value(6.88137356157246,label = 'b')
den1 = x1*w1; den1.label = 'den1'
den2 = x2*w2; den2.label = 'den2'
sig = den1 + den2; sig.label = 'sum'
# cellbody output
n = sig + b;n.label = 'N'
# activation: n.tanh()
o = n.tanh();o.label = 'O'
draw_dot(o)

In [ ]:
## testing backward function

o.grad=1
o._backward()
# n._backward()
# sig._backward()
# den1._backward()
# den2._backward()
draw_dot(o)

In [ ]:
## manual grad initialize

o.grad = 1

# o = n.tanh
n.grad = 1 - o.data**2  #1 - tanh**2 here o is tan(n) so replaced

#starting chain rule
b.grad = n.grad * 1
sig.grad = n.grad * 1

# sigma (w*x)
den1.grad = sig.grad * 1
den2.grad = sig.grad * 1

#dendrite 1
x1.grad = den1.grad * w1.data
w1.grad = den1.grad * x1.data

#dendrite 2
x2.grad = den2.grad * w2.data
w2.grad = den2.grad * x2.data

draw_dot(o)

In [ ]:
## Breaking down tanh

# Inputs no activation function
x1 = Value(2.0,label = 'x1')
x2 = Value(0.0,label = 'x2')
w1 = Value(-3.0,label = 'w1')
w2 = Value(1.0,label = 'w2')
b = Value(6.88137356157246,label = 'b')
den1 = x1*w1; den1.label = 'den1'
den2 = x2*w2; den2.label = 'den2'
sig = den1 + den2; sig.label = 'sum'
# cellbody output
n = sig + b;n.label = 'N'
# ---------
e = (2*n).exp();e.label = "xp()"
o = (e-1)/(e+1);o.label = 'O'
# ---------

# backward pass and vis
o.backward()
draw_dot(o)


# 

In [ ]:
## pytorch

import torch
x1 = torch.Tensor([2.0]).double() ; x1.requires_grad = True
x2 = torch.Tensor([0.0]).double() ; x2.requires_grad = True
w1 = torch.Tensor([-3.0]).double() ; w1.requires_grad = True
w2 = torch.Tensor([1.0]).double() ; w2.requires_grad = True
b = torch.Tensor([6.88137356157246]).double() ; b.requires_grad = True
n = x1*w1 + x2*w2 + b
o = torch.tanh(n)

print(o.data.item())
o.backward()

print("x1 : ",x1.grad.item())
print("x2 : ",x2.grad.item())
print("w1 : ",w1.grad.item())
print("w2 : ",w2.grad.item())
print("b : ",b.grad.item())

In [ ]:
## Neural net


class Neuron:
    def __init__(self,nin):
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1,1))

    def __call__(self,x):
        act = sum((wi*xi for wi,xi in zip(self.w,x)),self.b)
        out = act.tanh()
        return out

    def parameters(self):
        return self.w + [self.b]



class Layer:
    def __init__(self,nin,nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]

    def __call__(self,x):
        out = [n(x) for n in self.neurons]
        return out

    def parameters(self):
        p = []
        for i in self.neurons:
            p+=i.parameters()
        return p



class MLP:
    def __init__(self,nin,nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i],sz[i+1]) for i in range(len(nouts))]

    def  __call__ (self,x):
        for layer in self.layers:
            x = layer(x)
        return x[0] if len(x) == 1 else x

    def parameters(self):
        p = []
        for i in self.layers:
            p+=i.parameters()
        return p




x = [2.0, 3.0, -9.0]
n = MLP(3,[4,4,1])
n(x)

In [ ]:
n.parameters()

In [ ]:
draw_dot(n(x))

In [ ]:
xs = [[2.0 ,3.0 ,-1.0],[3.0 ,-1.0 ,0.5],[0.5 ,1.0 ,1.0],[1.0, 1.0,-1.0]]
ys = [1.0,-1.0,-1.0,1.0]
n = MLP(3,[4,4,1])


In [ ]:
for i in range(20):
    # forward pass
    y_pred = [n(x) for x in xs]
    loss = sum((yi - yg)**2 for yg,yi in zip(ys,y_pred))
    print(loss)

    # backward pass
    for p in n.parameters():
        p.grad = 0.0 ## zero grad
    loss.backward()

    # updates
    for p in n.parameters():
        p.data += -0.05 * p.grad
        
print("\n",y_pred)


In [ ]:
n.layers[0].neurons[0].w[0].data

In [ ]:
n.layers[0].neurons[0].w[0].grad

In [ ]:
for p in n.parameters():
    p.data += -0.1 * p.grad

In [ ]:
n.layers[0].neurons[0].w[0].data